<h3>Checkings</h3>

<p>dns lookups</p>

```bash
nslookup kshyst.ir
dig kshyst.ir
```

<p>check nginx</p>

```bash
sudo systemctl status nginx
sudo nginx -t
sudo tail -f /var/log/nginx/error.log
```

<p>check gunicorn</p>
```bash
sudo systemctl status myproject
```



<h3>Create a Gunicorn Systemd Service File:</h3>


```bash
sudo nano /etc/systemd/system/gunicorn.service
```
<pre><code>
[Unit]
Description=gunicorn daemon for myproject
After=network.target

[Service]
User=yourusername
Group=yourusername
WorkingDirectory=/path/to/your/project
ExecStart=/path/to/your/project/venv/bin/gunicorn --workers 3 --bind unix:/path/to/your/project/myproject.sock myproject.wsgi:application

[Install]
WantedBy=multi-user.target
</code></pre>


<p>then start the service</p>
    
```bash
sudo systemctl start myproject
sudo systemctl enable myproject
```



<h3>install nginx</h3>

```bash
sudo apt install nginx
```

```bash
sudo nano /etc/nginx/sites-available/myproject
```
<pre><code>
server {
    listen 80;
    server_name your_domain_or_IP;

    location = /favicon.ico { access_log off; log_not_found off; }
    location /static/ {
        root /path/to/your/project;
    }
    
    location /media/ {
        root /path/to/your/project;
    }

    location / {
        include proxy_params;
        proxy_pass http://unix:/path/to/your/project/myproject.sock;
    }
}

</code></pre>

<h3>Verify Nginx User:</h3>

```bash
sudo usermod -a -G kshyst www-data
```

<h3>Enable the New Nginx Configuration:</h3>

```bash
sudo ln -s /etc/nginx/sites-available/myproject /etc/nginx/sites-enabled
```

<h3>Test and Restart Nginx:</h3>

```bash
sudo nginx -t
sudo systemctl daemon-reload
sudo systemctl restart myproject
sudo systemctl restart nginx
```

<h3>Firewall</h3>

```bash
sudo ufw allow 'Nginx Full'
```

<h3>Logging errors of nginx</h3>

```bash
sudo tail -f /var/log/nginx/error.log
```

<h3>Creating name servers</h3>

```bash
sudo apt update
sudo apt install bind9 bind9utils bind9-doc
```

<p>add zone configurations to your domain</p>

```bash
sudo nano /etc/bind/named.conf.local
```

<pre><code>
zone "kshyst.ir" {
    type master;
    file "/etc/bind/zones/db.kshyst.ir";
};

zone "0.168.192.in-addr.arpa" {
    type master;
    file "/etc/bind/zones/db.192.168.0";
};

</code></pre>

<p>Create the zones folder</p>

```bash
sudo mkdir /etc/bind/db.kshyst.ir
```

<p>Configure BIND</p>

```bash
sudo nano /etc/bind/zones/db.kshyst.ir
```

<pre><code>
$TTL 86400
@   IN  SOA  ns1.kshyst.ir. admin.kshyst.ir. (
            2024072501 ; Serial
            3600       ; Refresh
            1800       ; Retry
            1209600    ; Expire
            86400 )    ; Minimum TTL
;
@   IN  NS  ns1.kshyst.ir.
@   IN  NS  ns2.kshyst.ir.
;
@   IN  A   <Your Server's IP Address>
ns1 IN  A   <Your First DNS Server's IP Address>
ns2 IN  A   <Your Second DNS Server's IP Address>

</code></pre>

<p>remove ns2 incase of 1 ip address</p>

```bash
sudo nano /etc/bind/zones/db.192.168.0
```

<pre><code>
$TTL 86400
@   IN  SOA  ns1.kshyst.ir. admin.kshyst.ir. (
            2024072501 ; Serial
            3600       ; Refresh
            1800       ; Retry
            1209600    ; Expire
            86400 )    ; Minimum TTL
;
@   IN  NS  ns1.kshyst.ir.
@   IN  NS  ns2.kshyst.ir.
;
1   IN  PTR kshyst.ir.

</code></pre>

<h3>Setup and test</h3>

```bash
sudo named-checkconf
sudo named-checkzone kshyst.ir /etc/bind/zones/db.kshyst.ir

sudo systemctl restart bind9
```

<p>Verify Setup</p>

```bash
dig @localhost kshyst.ir
```

<p>remember to add the new domain to allowed hosts in settings.py</p>

<h3>Add SSL</h3>

<p>stop nginx</p>

```bash
sudo systemctl stop nginx
```

<p>get cloudflare api token</p>
<pre>
Go to the Cloudflare dashboard.
Navigate to "My Profile" -> "API Tokens".
Create a new API token with the "Zone:DNS
" permission for the specific zone (domain) kshyst.ir.
</pre>

<p>install cerbot</p>

```bash
sudo apt update
sudo apt install certbot python3-certbot-nginx
sudo apt-get install python3-certbot-dns-cloudflare
```

<p>create cloudflare credentials</p>

```bash
sudo nano /etc/letsencrypt/cloudflare.ini
```

<code>dns_cloudflare_api_token = YOUR_CLOUDFLARE_API_TOKEN</code>

```bash
sudo chmod 600 /etc/letsencrypt/cloudflare.ini
```

<p>run cerbot</p>

```bash
sudo certbot certonly --dns-cloudflare --dns-cloudflare-credentials /etc/letsencrypt/cloudflare.ini -d kshyst.ir
```

<p>check if the certificate files are created</p>

```bash
sudo ls -l /etc/letsencrypt/live/kshyst.ir/
```

<p>the nginx configuration file should look like this</p>

```bash
sudo nano /etc/nginx/sites-available/myproject
```

<pre>
<code>
server {
    listen 80;
    server_name kshyst.ir;

    location / {
        proxy_pass http://unix:/home/kshyst/Portfolio/MyPortfolio.sock;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }

    location /static/ {
        alias /home/kshyst/Portfolio/static/;
    }

    location /media/ {
        alias /home/kshyst/Portfolio/media/;
    }

    # Redirect all HTTP requests to HTTPS
    if ($scheme != "https") {
        return 301 https://$host$request_uri;
    }
}

server {
    listen 443 ssl;
    server_name kshyst.ir;

    ssl_certificate /etc/letsencrypt/live/kshyst.ir/fullchain.pem;
    ssl_certificate_key /etc/letsencrypt/live/kshyst.ir/privkey.pem;
    include /etc/letsencrypt/options-ssl-nginx.conf;
    ssl_dhparam /etc/letsencrypt/ssl-dhparams.pem;

    location / {
        proxy_pass http://unix:/home/kshyst/Portfolio/MyPortfolio.sock;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }

    location /static/ {
        alias /home/kshyst/Portfolio/static/;
    }

    location /media/ {
        alias /home/kshyst/Portfolio/media/;
    }
}

</code>
</pre>

<p>now restart nginx to check if it works</p>

```bash
sudo systemctl start nginx
```


<h3>Setup Postgresql</h3>

<p>download dependencies</p>

```bash
sudo apt update
sudo apt install postgresql postgresql-contrib
sudo apt install libpq-dev
```

<p>activate virtual environment and install psycopg2</p>

```bash
source /path/to/your/venv/bin/activate
pip install psycopg2-binary
```

<p>Create a PostgreSQL Database and User</p>

```bash
sudo -i -u postgres
psql
```

<pre>
CREATE DATABASE mydatabase;
CREATE USER myuser WITH PASSWORD 'mypassword';
ALTER ROLE myuser SET client_encoding TO 'utf8';
ALTER ROLE myuser SET default_transaction_isolation TO 'read committed';
ALTER ROLE myuser SET timezone TO 'UTC';
GRANT ALL PRIVILEGES ON DATABASE mydatabase TO myuser;
\q
exit
</pre>

<p>update django's settings.py</p>

<pre>
<code>
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'mydatabase',
        'USER': 'myuser',
        'PASSWORD': 'mypassword',
        'HOST': 'localhost',
        'PORT': '5432',
    }
}
</code>
</pre>



<h3>Migrate the Database</h3>

```bash
python3 manage.py makemigrations
python3 manage.py migrate
```

<p>restart everything</p>

```bash
sudo systemctl daemon-reload
sudo systemctl restart myproject
sudo systemctl restart nginx
```

